In [355]:
import os
import random
import shutil
import re
import pandas as pd
import numpy as np

## clean* list of all files with no duplicates
the files have a lot of duplicates. remove those

input: 'rer_all_audio_files_list_upd.csv' from nuria

output: a pandas df 'no_duplicates'

In [358]:
data=pd.read_csv('rer_all_audio_files_list_upd.csv')

data.drop_duplicates(inplace=True) #remove duplicates in place
data.dropna(subset=['student_id'], inplace=True) #remove NaN subjects

duplicates = data[data.duplicated(subset=['student_id', 'task_name', 'item_name', 'semester'], keep=False)]

duplicates=duplicates.sort_values(by=['student_id', 'task_name', 'item_name', 'semester'])

#duplicates.to_csv('rer_duplicate_items.csv', index=False)

# Get rows of df1 which are not in df2
no_duplicates = data.merge(duplicates, how='outer', indicator=True).query("_merge == 'left_only'").drop('_merge', axis=1)

#no_duplicates
#no_duplicates.to_csv('no_duplicates.csv', index=False)

# Split the strings in col1 based on ':'
no_duplicates[['task', 'new_col2']] = no_duplicates['task_name'].str.split(':', expand=True)

# Split the strings in new_col2 based on '('
no_duplicates[['task', 'new_col3']] = no_duplicates['task'].str.split('(', expand=True)

# Drop the unwanted columns
no_duplicates.drop(['new_col2', 'new_col3'], axis=1, inplace=True)

# Replace all '_' and ' ' characters in col1 with '' and also remove Practice and Items
no_duplicates['task'] = no_duplicates['task'].str.replace('-', '').str.replace(' ', '').str.replace('Items', '').str.replace('Practice', '')


# clean the updated files list assessment by assessment
the files also have incopnsistent naming conventions and bad files that cannot be linked to scores. remove those

outputs: 4 *_clean_clean.csv cleaned file lists for each assessment for the kaggle challenge

## non word repetition

In [360]:
# Filter rows of df where task is 'NonwordRepetition', 'SentenceRepetition', 'Deletion', or 'Blending'
#new_df = no_duplicates[no_duplicates['task'].isin(['NonwordRepetition', 'SentenceRepetition', 'Deletion', 'Blending'])]
working_df = no_duplicates[no_duplicates['task'].isin(['NonwordRepetition'])]
# Split the strings in col1 based on '_'
working_df['item'] = working_df['item_name'].str.split('_', expand=True)[2]
working_df['assessment_season'] = working_df['assessment_name'].str.extract('(Fall|Winter|End)')
working_df['assessment_season'] = working_df['assessment_season'].str.lower()
working_df['semester_season'] = working_df['semester'].str.extract('(fall|winter|spring|end)')
working_df['true_season'] = working_df['semester_season'].fillna(working_df['assessment_season']) #combine semester and assessment season columns
working_df = working_df[~working_df['item'].isin(['S', 'A'])] #remove where item is 'S' or 'A' these don't line up with scores
where_nwr_bad=working_df['item'].isin(['0013','0009', '0003', '0006', '0027', '0024', '0025', '0018', '0001','0017', '0015', '0016','9003', '9002' ])
working_df['new_item']= working_df[where_nwr_bad]['item_name'].str.split('_', expand=True)[3]
working_df['item'] = working_df['item'].replace(['0013','0009', '0003', '0006', '0027', '0024', '0025', '0018', '0001','0017', '0015', '0016','9003', '9002'], np.nan)
working_df['item_name'] = working_df['item'].fillna(working_df['new_item']) #combine semester and assessment season columns
working_df = working_df[~working_df['item_name'].isin(['a'])] #remove where item is 'S' or 'A' these don't line up with scores
working_df['year'] = working_df['assessment_name'].str.extract('(2018|2019|2020|2021|2022|2023)')

#load all_files for creating the folders column
all_files=pd.read_csv('all_files.csv',header=None,names=['folder','audio_string'])
# merge the two dataframes on the 'key' column
working_df = pd.merge(working_df, all_files, on='audio_string', how='left')

col_task = working_df.pop('task')
col_grade = working_df.pop('grade')
col_audio_string = working_df.pop('audio_string')
col_student_id = working_df.pop('student_id')
col_true_season = working_df.pop('true_season')
col_folder = working_df.pop('folder')

# insert columns B and D at the first position
working_df.insert(0, 'task', col_task)
working_df.insert(0, 'folder', col_folder)
working_df.insert(0, 'true_season', col_true_season)
working_df.insert(0, 'student_id', col_student_id)
working_df.insert(0, 'audio_string', col_audio_string)
working_df.insert(0, 'grade', col_grade)

working_df.to_csv('NWR_clean_clean.csv', index=False)

/tmp/ipykernel_31867/144562154.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  working_df['item'] = working_df['item_name'].str.split('_', expand=True)[2]
/tmp/ipykernel_31867/144562154.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  working_df['assessment_season'] = working_df['assessment_name'].str.extract('(Fall|Winter|End)')
/tmp/ipykernel_31867/144562154.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

## Sentence rep

In [383]:
# Filter rows of df where task is 'NonwordRepetition', 'SentenceRepetition', 'Deletion', or 'Blending'
#new_df = no_duplicates[no_duplicates['task'].isin(['NonwordRepetition', 'SentenceRepetition', 'Deletion', 'Blending'])]
working_df = no_duplicates[no_duplicates['task'].isin(['SentenceRepetition'])]

# Split the strings in col1 based on '_'


working_df['assessment_season'] = working_df['assessment_name'].str.extract('(Fall|Winter|End)')
working_df['assessment_season'] = working_df['assessment_season'].str.lower()
working_df['semester_season'] = working_df['semester'].str.extract('(fall|winter|spring|end)')
working_df['true_season'] = working_df['semester_season'].fillna(working_df['assessment_season']) #combine semester and assessment season columns

working_df['item_name'] = working_df['item_name'].str.replace('_v2', '')
working_df['item_name'] = working_df['item_name'].str.replace('_012L', '')
working_df['item_name'] = working_df['item_name'].str.replace('_0L', '')
working_df['item_name'] = working_df['item_name'].str.replace('_01', '')
working_df['item_name'] = working_df['item_name'].str.replace('_0', '')
working_df['item_name'] = working_df['item_name'].str.replace('_1', '')
working_df['item_name'] = working_df['item_name'].str.replace('_2', '')
working_df['item_name'] = working_df['item_name'].str.replace('5', '')

working_df['item_name'] = working_df['item_name'].str.split('_').str[-1]
# working_df['item'] = working_df['item_name'].str.split('_', expand=True)[2]
working_df = working_df[~working_df['item_name'].isin(['U', 'U2', 'Diego', 'Practice1', 'Practice2' ])] #remove where item is 'S' or 'A' these don't line up with scores
working_df['year'] = working_df['date'].str.extract('(2018|2019|2020|2021|2022|2023)')

# where_nwr_bad=working_df['item'].isin(['0013','0009', '0003', '0006', '0027', '0024', '0025', '0018', '0001','0017', '0015', '0016','9003', '9002' ])
# working_df['new_item']= working_df[where_nwr_bad]['item_name'].str.split('_', expand=True)[3]
# working_df['item'] = working_df['item'].replace(['0013','0009', '0003', '0006', '0027', '0024', '0025', '0018', '0001','0017', '0015', '0016','9003', '9002'], np.nan)
# working_df['true_item'] = working_df['item'].fillna(working_df['new_item']) #combine semester and assessment season columns
all_files=pd.read_csv('all_files.csv',header=None,names=['folder','audio_string'])
working_df = pd.merge(working_df, all_files, on='audio_string', how='left')

col_task = working_df.pop('task')
col_grade = working_df.pop('grade')
col_audio_string = working_df.pop('audio_string')
col_student_id = working_df.pop('student_id')
col_true_season = working_df.pop('true_season')
col_folder = working_df.pop('folder')

# insert columns B and D at the first position
working_df.insert(0, 'task', col_task)
working_df.insert(0, 'folder', col_folder)
working_df.insert(0, 'true_season', col_true_season)
working_df.insert(0, 'student_id', col_student_id)
working_df.insert(0, 'audio_string', col_audio_string)
working_df.insert(0, 'grade', col_grade)

working_df.to_csv('SRT_clean_clean.csv', index=False)

/tmp/ipykernel_31867/3038809114.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  working_df['assessment_season'] = working_df['assessment_name'].str.extract('(Fall|Winter|End)')
/tmp/ipykernel_31867/3038809114.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  working_df['assessment_season'] = working_df['assessment_season'].str.lower()
/tmp/ipykernel_31867/3038809114.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer

## Blending

In [362]:
# Filter rows of df where task is 'NonwordRepetition', 'SentenceRepetition', 'Deletion', or 'Blending'
#new_df = no_duplicates[no_duplicates['task'].isin(['NonwordRepetition', 'SentenceRepetition', 'Deletion', 'Blending'])]
working_df = no_duplicates[no_duplicates['task'].isin(['Blending'])]

# Split the strings in col1 based on '_'


working_df['assessment_season'] = working_df['assessment_name'].str.extract('(Fall|Winter|End)')
working_df['assessment_season'] = working_df['assessment_season'].str.lower()
working_df['semester_season'] = working_df['semester'].str.extract('(fall|winter|spring|end)')
working_df['true_season'] = working_df['semester_season'].fillna(working_df['assessment_season']) #combine semester and assessment season columns
working_df['item_name'] = working_df['item_name'].str.replace('_0L', '')
working_df['item_name'] = working_df['item_name'].str.replace('_0', '')

# working_df['item_name'] = working_df['item_name'].str.replace('_v2', '')
# working_df['item_name'] = working_df['item_name'].str.replace('_012L', '')
# working_df['item_name'] = working_df['item_name'].str.replace('_0L', '')
# working_df['item_name'] = working_df['item_name'].str.replace('_01', '')
# working_df['item_name'] = working_df['item_name'].str.replace('_0', '')
# working_df['item_name'] = working_df['item_name'].str.replace('_1', '')
# working_df['item_name'] = working_df['item_name'].str.replace('_2', '')
# working_df['item_name'] = working_df['item_name'].str.replace('5', '')

working_df['item_name'] = working_df['item_name'].str.split('_').str[-1]
working_df['item_name'] = working_df['item_name'].str.lower()

# # working_df['item'] = working_df['item_name'].str.split('_', expand=True)[2]
working_df = working_df[~working_df['item_name'].isin(['d', 'u', 'p'])] #remove where item is 'S' or 'A' these don't line up with scores
working_df['year'] = working_df['assessment_name'].str.extract('(2018|2019|2020|2021|2022|2023)')

# # where_nwr_bad=working_df['item'].isin(['0013','0009', '0003', '0006', '0027', '0024', '0025', '0018', '0001','0017', '0015', '0016','9003', '9002' ])
# # working_df['new_item']= working_df[where_nwr_bad]['item_name'].str.split('_', expand=True)[3]
# # working_df['item'] = working_df['item'].replace(['0013','0009', '0003', '0006', '0027', '0024', '0025', '0018', '0001','0017', '0015', '0016','9003', '9002'], np.nan)
# # working_df['true_item'] = working_df['item'].fillna(working_df['new_item']) #combine semester and assessment season columns
all_files=pd.read_csv('all_files.csv',header=None,names=['folder','audio_string'])
working_df = pd.merge(working_df, all_files, on='audio_string', how='left')

col_task = working_df.pop('task')
col_grade = working_df.pop('grade')
col_audio_string = working_df.pop('audio_string')
col_student_id = working_df.pop('student_id')
col_true_season = working_df.pop('true_season')
col_folder = working_df.pop('folder')

# insert columns B and D at the first position
working_df.insert(0, 'task', col_task)
working_df.insert(0, 'folder', col_folder)
working_df.insert(0, 'true_season', col_true_season)
working_df.insert(0, 'student_id', col_student_id)
working_df.insert(0, 'audio_string', col_audio_string)
working_df.insert(0, 'grade', col_grade)

working_df.to_csv('BLE_clean_clean.csv', index=False)

/tmp/ipykernel_31867/403177495.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  working_df['assessment_season'] = working_df['assessment_name'].str.extract('(Fall|Winter|End)')
/tmp/ipykernel_31867/403177495.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  working_df['assessment_season'] = working_df['assessment_season'].str.lower()
/tmp/ipykernel_31867/403177495.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

## Deletion

In [372]:
# Filter rows of df where task is 'NonwordRepetition', 'SentenceRepetition', 'Deletion', or 'Blending'
#new_df = no_duplicates[no_duplicates['task'].isin(['NonwordRepetition', 'SentenceRepetition', 'Deletion', 'Blending'])]
working_df = no_duplicates[no_duplicates['task'].isin(['Deletion'])]

# Split the strings in col1 based on '_'


working_df['assessment_season'] = working_df['assessment_name'].str.extract('(Fall|Winter|End)')
working_df['assessment_season'] = working_df['assessment_season'].str.lower()
working_df['semester_season'] = working_df['semester'].str.extract('(fall|winter|spring|end)')
working_df['true_season'] = working_df['semester_season'].fillna(working_df['assessment_season']) #combine semester and assessment season columns


working_df['item_name'] = working_df['item_name'].str.replace('_v2', '')
working_df['item_name'] = working_df['item_name'].str.replace('_012L', '')
working_df['item_name'] = working_df['item_name'].str.replace('_0L', '')
working_df['item_name'] = working_df['item_name'].str.replace('_01', '')
working_df['item_name'] = working_df['item_name'].str.replace('_0', '')
working_df['item_name'] = working_df['item_name'].str.replace('_1', '')
working_df['item_name'] = working_df['item_name'].str.replace('_2', '')



working_df['item_name'] = working_df['item_name'].str.split('_').str[-1]
working_df['item_name'] = working_df['item_name'].str.lower()
working_df['item_name'] = working_df['item_name'].str.replace('1', '')
working_df['item_name'] = working_df['item_name'].str.replace('2', '')

# # # working_df['item'] = working_df['item_name'].str.split('_', expand=True)[2]
working_df = working_df[~working_df['item_name'].isin(['d', 'u', 'p', '9'])] #remove where item is 'S' or 'A' these don't line up with scores
working_df['year'] = working_df['assessment_name'].str.extract('(2018|2019|2020|2021|2022|2023)').astype(str)

# # # where_nwr_bad=working_df['item'].isin(['0013','0009', '0003', '0006', '0027', '0024', '0025', '0018', '0001','0017', '0015', '0016','9003', '9002' ])
# # # working_df['new_item']= working_df[where_nwr_bad]['item_name'].str.split('_', expand=True)[3]
# # # working_df['item'] = working_df['item'].replace(['0013','0009', '0003', '0006', '0027', '0024', '0025', '0018', '0001','0017', '0015', '0016','9003', '9002'], np.nan)
# # # working_df['true_item'] = working_df['item'].fillna(working_df['new_item']) #combine semester and assessment season columns
all_files=pd.read_csv('all_files.csv',header=None,names=['folder','audio_string'])
working_df = pd.merge(working_df, all_files, on='audio_string', how='left')

col_task = working_df.pop('task')
col_grade = working_df.pop('grade')
col_audio_string = working_df.pop('audio_string')
col_student_id = working_df.pop('student_id')
col_true_season = working_df.pop('true_season')
col_folder = working_df.pop('folder')

# insert columns B and D at the first position
working_df.insert(0, 'task', col_task)
working_df.insert(0, 'folder', col_folder)
working_df.insert(0, 'true_season', col_true_season)
working_df.insert(0, 'student_id', col_student_id)
working_df.insert(0, 'audio_string', col_audio_string)
working_df.insert(0, 'grade', col_grade)

working_df.to_csv('DEL_clean_clean.csv', index=False)

/tmp/ipykernel_31867/932121695.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  working_df['assessment_season'] = working_df['assessment_name'].str.extract('(Fall|Winter|End)')
/tmp/ipykernel_31867/932121695.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  working_df['assessment_season'] = working_df['assessment_season'].str.lower()
/tmp/ipykernel_31867/932121695.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

# Unify Scores
match up the given scores with the cleaned updated files lists

outputs: 4 *_unified.csv files with all of the non-duplicate identifiable files with all info parsed and scores added

In [413]:
assessments=['NWR','DEL','BLE','SRT']
for a in assessments:
    scores_df=pd.read_csv(f'{a}_clean.csv')
    scores_df.rename(columns={'Student.Tracking.Id': 'student'}, inplace=True)
    # Extract the season from uniqueid
    scores_df['season'] = scores_df['uniqueid'].str.extract('(fall|spring|winter)')
    scores_df['year'] = scores_df['uniqueid'].str.extract('(18|19|20|21|22|23)')
    scores_df['year'] = scores_df['year'].apply(lambda x: pd.to_datetime(str(x), format='%y').strftime('%Y'))
    # Replace 'winter' with 'spring' in col1
    scores_df['season'] = scores_df['season'].replace('spring', 'winter')
    scores_df = scores_df.rename(columns={'Grade': 'grade'})
    scores_df = scores_df.rename(columns={'season': 'true_season'})
    scores_df = scores_df.rename(columns={'student': 'student_id'})
    scores_df_melt = scores_df.melt(id_vars=['uniqueid', 'student_id', 'grade', 'true_season', 'year'], var_name='item_name', value_name='score')
    
    #remove NaNs
    scores_df_melt = scores_df_melt[scores_df_melt['score'].notna()]
    
    files_df=pd.read_csv(f'{a}_clean_clean.csv')
    files_scores = pd.merge(files_df.astype(str), scores_df_melt.astype(str), on=['student_id', 'grade', 'true_season','item_name','year'], how='left')
    # get a dataframe where all of the values of the column 'score' are NaN
    files_scores_nan = files_scores[files_scores['score'].isna()]
    #files_scores_nan
    # get a dataframe where all of the values of the column 'score' are not NaN
    files_scores_not_nan = files_scores[files_scores['score'].notna()]
    
    files_scores_not_nan = files_scores_not_nan.drop(['assessment_season','semester_season'], axis=1)

    #REMOVE ALL DUPLCIATES FOR NOW
    files_scores_not_nan.drop_duplicates(subset=['audio_string'],inplace=True,keep=False)
    
    files_scores_not_nan.to_csv(f'{a}_unified.csv', index=False)